In [1]:
# Import the necessary libraries
import sys
import os
import pandas as pd
import warnings

# Add the src folder to the path
sys.path.append(os.path.abspath('..'))
warnings.filterwarnings("ignore", category=FutureWarning)

# Import the classes
from src.data_tool.connect_wrds import get_wrds_data, get_security_daily
from src.data_tool.yahoo_finance import yahoo_finance
from src.data_tool.ploygon import polygon
from src.data_tool.financial_datasets import FinancialDatasets
from src.data_tool.connect_db import ConnectDB
from src.data_tool.create_table import TableCreator

# Initialize the classes
yf = yahoo_finance()
pl = polygon()
fd = FinancialDatasets()

db =ConnectDB()
engine = db.get_engine()


FINANCIAL_DATASETS_API_KEY is set
YAHOO_CONSUMER_KEY and YAHOO_CONSUMER_SECRET are set
YAHOO_APP_ID is set
POLYGON_API_KEY is set
DB_USER and DB_PASSWORD are set


In [2]:
def get_database_size(db_instance, database_name):
    """
    Retrieves the total size of the database.
    """
    sql_statement = f"""
    SELECT table_schema "database_name", 
           sum(data_length + index_length) / 1024 / 1024 "database_size_mb" 
    FROM information_schema.TABLES 
    WHERE table_schema = '{database_name}'
    GROUP BY table_schema;
    """
    result = db_instance.execute_sql(sql_statement)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    return df

database_name = 'Elite_Traders' # Replace with your database name if different
database_size_df = get_database_size(db, database_name)
print(database_size_df)

KeyboardInterrupt: 

In [ ]:
def get_table_sizes(db_instance):
    """
    Retrieves the size of each table in the database.
    """
    sql_statement = "SHOW TABLE STATUS"
    result = db_instance.execute_sql(sql_statement)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    size_df = df[['Name', 'Data_length', 'Index_length', 'Data_free']]
    size_df['Total_size'] = size_df['Data_length'] + size_df['Index_length']
    size_df['Data_length_MB'] = size_df['Data_length'] / 1024 / 1024
    size_df['Index_length_MB'] = size_df['Index_length'] / 1024 / 1024
    size_df['Data_free_MB'] = size_df['Data_free'] / 1024 / 1024
    size_df['Total_size_MB'] = size_df['Total_size'] / 1024 / 1024
    return size_df

table_sizes_df = get_table_sizes(db)
print(table_sizes_df)

In [3]:
db.show_tables()

,Tables
0,capital_structure_debt
1,capital_structure_equity
2,capital_structure_summary
3,financial_ratios
4,fundamentals_annual
5,fundamentals_quarterly
6,identifiers
7,key_developments
8,security_daily


In [4]:
db.read_table('key_developments')

KeyboardInterrupt: 

In [8]:
tickers = ['AAPL']
start_date = '2023-01-01'
end_date = '2023-12-31'

In [9]:
data = yf.get_data(tickers, start_date, end_date)

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
db.create_table('test', df)